In [12]:
import random
import matplotlib.pyplot as plt
import numpy as np

In [13]:
LIGNES = 6
COLONNES = 7

CASES_POUR_GAGNER = 4
CASE_VIDE = "."

PION_JOUEUR_1 = "X"
PION_JOUEUR_2 = "O"

In [14]:
def Quadruplet():
  liste = np.empty((0,4,2))
  for i in range(0,LIGNES):
    for j in range(0,COLONNES):
      if i+3 < LIGNES :
        ligne = np.array([[[i,j],[i+1,j],[i+2,j],[i+3,j]]])
        liste = np.append(liste,ligne,axis=0)
      if j+3 < COLONNES :
        colonne = np.array([[[i,j],[i,j+1],[i,j+2],[i,j+3]]])
        liste = np.append(liste,colonne,axis=0)
      if i+3 < LIGNES and j+3 < COLONNES :
        diagonal = np.array([[[i,j],[i+1,j+1],[i+2,j+2],[i+3,j+3]]])
        liste = np.append(liste,diagonal,axis=0)
      if i-3 >= 0 and j-3 >= 0 :
        diagonal = np.array([[[i,j],[i-1,j-1],[i-2,j-2],[i-3,j-3]]])
        liste = np.append(liste,diagonal,axis=0)
  return liste

In [15]:
tab = Quadruplet().shape
print("Nombre de quadruplet avec",str(LIGNES),"ligne et",str(COLONNES),"colonne :",tab[0])

Nombre de quadruplet avec 6 ligne et 7 colonne : 69


In [16]:
class Plateau(object):

    def __init__(self, nb_lignes=1, nb_colonnes=1, val_def=None):
        self._lignes = []
        self._val_def = val_def

        self._cases_vides = nb_colonnes * nb_lignes

        for i in range(nb_lignes):
            self._lignes.append([val_def] * nb_colonnes)

    def afficher(self):
        for i in range(self.nb_lignes):
            for j in range(self.nb_colonnes):
                print(self._lignes[i][j], end=' ')
            print()

    @property
    def nb_lignes(self):
        return len(self._lignes)

    @property
    def nb_colonnes(self):
        return len(self._lignes[0])

    @property
    def nb_cases_vides(self):
        return self._cases_vides

    def clean(self):
        for i in range(self.nb_lignes):
            for j in range(self.nb_colonnes):
                self._lignes[i][j] = self._val_def

        self._cases_vides = self.nb_colonnes * self.nb_lignes

    @property
    def is_clean(self):
        return self._cases_vides == self.nb_lignes * self.nb_colonnes

    @property
    def is_full(self):
        return self._cases_vides == 0

    def valeur(self, ligne, colonne):
        return self._lignes[ligne][colonne]

    def fill_case(self, ligne, colonne, valeur):
        if valeur == self._val_def and self._lignes[ligne][colonne] != self._val_def:
            self._cases_vides += 1

        elif valeur != self._val_def and self._lignes[ligne][colonne] == self._val_def:
            self._cases_vides -= 1

        self._lignes[ligne][colonne] = valeur

    def clean_case(self, ligne, colonne):
        self.fill_case(ligne, colonne, self._val_def)

    def case_is_clean(self, ligne, colonne):
        return self._lignes[ligne][colonne] == self._val_def

    def Quadruplet():
        liste = np.empty((0,4,2))
        for i in range(0,LIGNES):
            for j in range(0,COLONNES):
                if i+3 < LIGNES :
                    ligne = np.array([[[i,j],[i+1,j],[i+2,j],[i+3,j]]])
                    liste = np.append(liste,ligne,axis=0)
                if j+3 < COLONNES :
                    colonne = np.array([[[i,j],[i,j+1],[i,j+2],[i,j+3]]])
                    liste = np.append(liste,colonne,axis=0)
                if i+3 < LIGNES and j+3 < COLONNES :
                    diagonal = np.array([[[i,j],[i+1,j+1],[i+2,j+2],[i+3,j+3]]])
                    liste = np.append(liste,diagonal,axis=0)
                if i-3 >= 0 and j-3 >= 0 :
                    diagonal = np.array([[[i,j],[i-1,j-1],[i-2,j-2],[i-3,j-3]]])
                    liste = np.append(liste,diagonal,axis=0)
        return liste

In [17]:
class Coup(object):

    def __init__(self, ligne, colonne, pion):
        self.ligne = ligne
        self.colonne = colonne
        self.pion = pion

In [18]:
class Jeu(object):
    
    def __init__(self, nb_lignes=LIGNES, nb_colonnes=COLONNES, nbre_cases_gain=CASES_POUR_GAGNER):
        self._plateau = Plateau(nb_lignes, nb_colonnes, CASE_VIDE)
        self.nbre_cases_gain = nbre_cases_gain
        self.gagne = False

    @property
    def nb_lignes(self):
        return self._plateau.nb_lignes

    @property
    def nb_colonnes(self):
        return self._plateau.nb_colonnes

    @property
    def nul(self):
        return self._plateau.is_full and not self.gagne

    @property
    def fini(self):
        return self.gagne or self._plateau.is_full

    def afficher(self):
        self._plateau.afficher()

    def reinitialiser(self):
        self._plateau.clean()
        self.gagne = False
    
    @property
    def colonnes_jouables(self):
        colonnes_jouables = []
        for colonne in range(self._plateau.nb_colonnes):
            if self._plateau.case_is_clean(0, colonne):
                colonnes_jouables.append(colonne)
        return colonnes_jouables
    
    def has_won(self, dernier_coup):
        """
        Recherche si le dernier coup est gagnant
        :param dernier_coup:
        :return: True/False
        """
        # ----------------------------------------------------------------------------------------------------------------------------
        # Pas besoin de tester au dessus du dernier coup (cases vides forcément)

        # ----------------------------------------------------------------------------------------------------------------------------
        # Cases du dessous
        if dernier_coup.ligne + self.nbre_cases_gain <= self._plateau.nb_lignes:
            for ligne in range(dernier_coup.ligne, dernier_coup.ligne + self.nbre_cases_gain):
                if self._plateau.valeur(ligne, dernier_coup.colonne) != dernier_coup.pion:
                    break
            else:
                # Ce 'else' ne s'execute que si la boucle se termine sans 'break' : le jeu est alors gagné
                return True

        # ----------------------------------------------------------------------------------------------------------------------------
        # Cases horizontales
        recherche_droite = True
        recherche_gauche = True
        colonne_droite = dernier_coup.colonne + 1
        colonne_gauche = dernier_coup.colonne - 1
        nbre_de_cases_alignees = 1

        while True:
            if recherche_droite:
                if colonne_droite < self._plateau.nb_colonnes and self._plateau.valeur(dernier_coup.ligne, colonne_droite) == dernier_coup.pion:
                    nbre_de_cases_alignees += 1
                    if nbre_de_cases_alignees == self.nbre_cases_gain:
                        return True

                    colonne_droite += 1
                else:
                    recherche_droite = False

            if recherche_gauche:
                if colonne_gauche >= 0 and self._plateau.valeur(dernier_coup.ligne, colonne_gauche) == dernier_coup.pion:
                    nbre_de_cases_alignees += 1
                    if nbre_de_cases_alignees == self.nbre_cases_gain:
                        return True

                    colonne_gauche -= 1
                else:
                    recherche_gauche = False

            if not recherche_droite and not recherche_gauche:
                break

        # ----------------------------------------------------------------------------------------------------------------------------
        # Diagonales droites (/)
        recherche_haut = True
        recherche_bas = True
        colonne_droite = dernier_coup.colonne + 1
        colonne_gauche = dernier_coup.colonne - 1
        ligne_haut = dernier_coup.ligne - 1
        ligne_bas = dernier_coup.ligne + 1
        nbre_de_cases_alignees = 1

        while True:
            if recherche_haut:
                if colonne_droite < self._plateau.nb_colonnes and ligne_haut >= 0 and self._plateau.valeur(ligne_haut, colonne_droite) == dernier_coup.pion:
                    nbre_de_cases_alignees += 1
                    if nbre_de_cases_alignees == self.nbre_cases_gain:
                        return True

                    colonne_droite += 1
                    ligne_haut -= 1
                else:
                    recherche_haut = False

            if recherche_bas:
                if colonne_gauche >= 0 and ligne_bas < self._plateau.nb_lignes and self._plateau.valeur(ligne_bas, colonne_gauche) == dernier_coup.pion:
                    nbre_de_cases_alignees += 1
                    if nbre_de_cases_alignees == self.nbre_cases_gain:
                        return True

                    colonne_gauche -= 1
                    ligne_bas += 1
                else:
                    recherche_bas = False

            if not recherche_haut and not recherche_bas:
                break

        # ----------------------------------------------------------------------------------------------------------------------------
        # Diagonales gauches (\)
        recherche_haut = True
        recherche_bas = True
        colonne_droite = dernier_coup.colonne + 1
        colonne_gauche = dernier_coup.colonne - 1
        ligne_haut = dernier_coup.ligne - 1
        ligne_bas = dernier_coup.ligne + 1
        nbre_de_cases_alignees = 1

        while True:
            if recherche_haut:
                if colonne_gauche >= 0 and ligne_haut >= 0 and self._plateau.valeur(ligne_haut, colonne_gauche) == dernier_coup.pion:
                    nbre_de_cases_alignees += 1
                    if nbre_de_cases_alignees == self.nbre_cases_gain:
                        return True

                    colonne_gauche -= 1
                    ligne_haut -= 1
                else:
                    recherche_haut = False

            if recherche_bas:
                if colonne_droite < self._plateau.nb_colonnes and ligne_bas < self._plateau.nb_lignes and self._plateau.valeur(ligne_bas, colonne_droite) == dernier_coup.pion:
                    nbre_de_cases_alignees += 1
                    if nbre_de_cases_alignees == self.nbre_cases_gain:
                        return True

                    colonne_droite += 1
                    ligne_bas += 1
                else:
                    recherche_bas = False

            if not recherche_haut and not recherche_bas:
                break
        return False
    
    def jouer(self, colonne, pion):
        for ligne in range(self._plateau.nb_lignes - 1, -1, -1):
            if self._plateau.case_is_clean(ligne, colonne):
                coup = Coup(ligne, colonne, pion)

                self._plateau.fill_case(ligne, colonne, pion)

                self.gagne = self.has_won(coup)
                return coup



In [19]:
class JoueurHumain(object):
    
    def __init__(self, jeu, nom, pion):
        self.nom = nom
        self.pion = pion
        self._jeu = jeu

    def choix(self):
        while True:
            reponse = input("Où voulez-vous jouer ? \n (exit pour quitter) ")
            try:
                reponse = int(reponse)
            except ValueError:
                if reponse == 'exit':
                    return -1
            else:
                if reponse in self._jeu.colonnes_jouables:
                    return reponse
                else:
                    print('Veuillez saisir une valeur correcte')

In [20]:
class IaAleatoire(object):

    def __init__(self, jeu, nom, pion):
        self.nom = nom
        self.pion = pion
        self._jeu = jeu

    def choix(self):
        return random.randrange(len(self._jeu.colonnes_jouables))

In [21]:
class PartieConsole(object):
    def __init__(self, jeu):
        self._jeu = jeu
        self._joueur1 = None
        self._joueur2 = None
        self._joueur_courant = None

        # entêtes de colonne: c'est le numéro à saisir par le joueur pour placer son pion dans la colonne
        self._en_tete_colonne = [str(numero_colonne) for numero_colonne in range(self._jeu.nb_colonnes)]

    def _effacer_ecran(self):
        """
        Efface la console (1ère commande pour windows, 2ème pour linux)
        """
        os.system('cls' if os.name == 'nt' else 'clear')

    def _affiche_plateau(self):
        """
        Affiche le plateau de jeu : en tête + plateau
        """
        # affiche l'en-tête
        ncolonne = len(self._en_tete_colonne)
        for i in range(ncolonne):
            print(self._en_tete_colonne[i], end=' ')
        print()

        # affiche le jeu
        self._jeu.afficher()

        # affiche la séparation
        print()
        print("= " * ncolonne)

    def _recommencer(self):
        """
        Demande au joueur si il souhaite rejouer
        :return: True/False
        """
        while True:
            i = input('Voulez-vous rejouer ? o/n ')
            if i in ('o', 'n'):
                return i == 'o'
            else:
                print("Je n'ai pas bien compris...")

    def _nouvelle_partie(self):
        """
        Initialise une nouvelle partie
        """
        self._effacer_ecran()
        print('Bienvenue dans le jeu du puissance 4 !')

        nbre_joueurs_humains = self._demander_nbre_joueurs()

        if nbre_joueurs_humains == 0:
            profondeur = self._demander_profondeur_ia()
            if profondeur == -1:
                self._joueur1 = IaAleatoire(self._jeu, 'Random1', PION_JOUEUR_1)
                self._joueur2 = IaAleatoire(self._jeu, 'Random2', PION_JOUEUR_2)

        elif nbre_joueurs_humains == 1:
            nom = input('Pseudo du joueur ?')
            while nom == 'Random':
                print('Le nom du joueur doit être différent de "Random" !')
                nom = input('Pseudo du joueur ?')
            profondeur = self._demander_profondeur_ia()
            self._joueur1 = JoueurHumain(self._jeu, nom, PION_JOUEUR_1)
            if profondeur == -1:
                self._joueur2 = IaAleatoire(self._jeu, 'Random', PION_JOUEUR_2)

        elif nbre_joueurs_humains == 2:
            nom1 = input('Pseudo du joueur 1 ?')
            nom2 = input('Pseudo du joueur 2 ?')
            while nom1 == nom2:
                print('Le nom du joueur 2 doit être différent de celui du joueur 1 !')
                nom2 = input('Pseudo du joueur 2 ?')
            self._joueur1 = JoueurHumain(self._jeu, nom1, PION_JOUEUR_1)
            self._joueur2 = JoueurHumain(self._jeu, nom2, PION_JOUEUR_2)

        self._joueur_courant = self._joueur1
        self._jeu.reinitialiser()
        self._effacer_ecran()
        self._affiche_plateau()

    def _demander_nbre_joueurs(self):
        """
        Demande le nombre de joueurs humains à l'utilisateur
        :return: entier compris entre 0 et 2
        """
        while True:
            try:
                nbre_joueurs_humains = int(input('Nombre de joueurs humains (0/1/2) ?'))
            except ValueError:
                pass
            else:
                if nbre_joueurs_humains in [0, 1, 2]:
                    return nbre_joueurs_humains

    def _demander_profondeur_ia(self):
        """
        Demande la difficulté de l'IA à l'utilisateur (correspond à la profondeur de recherche des coups)
        :return: entier (-1 pour l'IA aléatoire, positif ou nulle pour MinMAx)
        """
        while True:
            try:
                profondeur = int(input("(-1) pour une IA aléatoire ?"))
            except ValueError:
                print('Veuillez saisir un nombre entier !')
            else:
                if profondeur < -1:
                    print('Veuillez saisir une valeur correcte !')
                    return self._demander_profondeur_ia()
                else:
                    return profondeur

    def jouer(self):
        """
        Lance une partie entre les deux joueurs
        """
        self._nouvelle_partie()

        while True:
            # affiche le joueur courant
            print("C'est", self._joueur_courant.nom, "qui joue avec les ", self._joueur_courant.pion, " !")

            # choix d'un emplacement de jeu par le joueur
            choix = self._joueur_courant.choix()
            if choix == -1:
                break

            # on joue le coup
            self._jeu.jouer(choix, self._joueur_courant.pion)
            self._effacer_ecran()
            self._affiche_plateau()

            # Préparation du tour de jeu suivant
            if self._jeu.gagne:
                print('Bravo', self._joueur_courant.nom, '! Tu as gagné !')
                if self._recommencer():
                    self._nouvelle_partie()
                else:
                    break

            elif self._jeu.nul:
                print("Fin de la partie, personne n'a gagné !")
                if self._recommencer():
                    self._nouvelle_partie()
                else:
                    break
            else:
                # inversion des joueurs
                if self._joueur_courant == self._joueur1:
                    self._joueur_courant = self._joueur2
                else:
                    self._joueur_courant = self._joueur1

In [22]:
puissance4 = Jeu()
partie = PartieConsole(puissance4)
partie.jouer()

Bienvenue dans le jeu du puissance 4 !
0 1 2 3 4 5 6 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 

= = = = = = = 
C'est Random1 qui joue avec les  X  !
0 1 2 3 4 5 6 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . X 

= = = = = = = 
C'est Random2 qui joue avec les  O  !
0 1 2 3 4 5 6 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . O . X 

= = = = = = = 
C'est Random1 qui joue avec les  X  !
0 1 2 3 4 5 6 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. X . . O . X 

= = = = = = = 
C'est Random2 qui joue avec les  O  !
0 1 2 3 4 5 6 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. O . . . . . 
. X . . O . X 

= = = = = = = 
C'est Random1 qui joue avec les  X  !
0 1 2 3 4 5 6 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. . . . . . . 
. O . . . . . 
. X . X O . X 

= = = = = = = 
C'est Random2 qui joue avec les  O  !
0 1 2 3